
# Meta-analysis of Experiments: Portfolio View

This notebook treats a **set of experiments** as a *portfolio* and shows how to
combine their results using **meta-analysis**.

Instead of looking at a single A/B test in isolation, we ask:

- What is the **typical effect size** across many experiments?
- How much **heterogeneity** is there between experiments?
- How are our conclusions biased if we only look at **statistically significant** wins?
- How can we build **partial-pooling / Bayesian-style shrinkage** for experiment effects?

We will:

1. Simulate a portfolio of experiments on the same metric.  
2. Compute per-experiment effects and p-values.  
3. Perform **fixed-effect** and **random-effects (DerSimonian–Laird)** meta-analysis.  
4. Visualize a **forest plot** of experiment estimates and the pooled effect.  
5. Show **publication bias** by only looking at significant experiments.  
6. Build a simple **Bayesian-style shrinkage** for experiment effects (partial pooling).


## 0) Setup

In [ ]:

from __future__ import annotations

from dataclasses import dataclass
from typing import List, Dict, Any

import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

plt.rcParams["figure.figsize"] = (8, 4.5)
plt.rcParams["axes.grid"] = True



## 1) Simulating a portfolio of experiments

We simulate a set of experiments that:

- All target the **same metric** (e.g., conversion rate).  
- Share a **global average treatment effect** \(\mu\).  
- Have **between-experiment heterogeneity** \(\tau\) (some experiments do better, some worse).  
- Have varying sample sizes and noise.

Model (log-odds formulation):

- Baseline control conversion: \(p_C\).  
- For experiment \(i\), draw a *true* log-odds treatment effect \(\theta_i \sim \mathcal{N}(\mu, \tau^2)\).  
- Treatment conversion probability:

\[
\text{logit}(p_{T,i}) = \text{logit}(p_C) + \theta_i.
\]

We then run a standard two-proportion experiment and estimate the **difference in conversion rates**.


In [ ]:

@dataclass
class PortfolioSimConfig:
    n_experiments: int = 50
    mean_logodds_effect: float = 0.05
    tau_logodds: float = 0.15
    baseline_rate: float = 0.10
    mean_n_per_group: int = 5000
    n_sd_per_group: int = 1000
    seed: int | None = 1234


def _logit(p: float) -> float:
    if p <= 0.0 or p >= 1.0:
        raise ValueError("p must be in (0,1) for logit.")
    return math.log(p / (1.0 - p))


def _inv_logit(z: float) -> float:
    return 1.0 / (1.0 + math.exp(-z))


def simulate_experiment_portfolio(config: PortfolioSimConfig) -> pd.DataFrame:
    rng = np.random.default_rng(config.seed)

    theta_true = rng.normal(
        loc=config.mean_logodds_effect,
        scale=config.tau_logodds,
        size=config.n_experiments,
    )

    logit_p_c = _logit(config.baseline_rate)

    records: list[Dict[str, Any]] = []

    for i in range(config.n_experiments):
        n_c = int(max(100, rng.normal(config.mean_n_per_group, config.n_sd_per_group)))
        n_t = int(max(100, rng.normal(config.mean_n_per_group, config.n_sd_per_group)))

        theta_i = float(theta_true[i])
        p_c = config.baseline_rate
        p_t = _inv_logit(logit_p_c + theta_i)

        conv_c = int(rng.binomial(n_c, p_c))
        conv_t = int(rng.binomial(n_t, p_t))

        rate_c = conv_c / n_c
        rate_t = conv_t / n_t
        effect_hat = rate_t - rate_c

        se_hat = math.sqrt(
            rate_c * (1.0 - rate_c) / n_c + rate_t * (1.0 - rate_t) / n_t
        )

        z_stat = effect_hat / se_hat if se_hat > 0 else float("nan")
        cdf = 0.5 * (1.0 + math.erf(z_stat / math.sqrt(2.0)))
        p_value = 2.0 * min(cdf, 1.0 - cdf)

        records.append(
            {
                "exp_id": i,
                "n_control": n_c,
                "n_treatment": n_t,
                "conv_control": conv_c,
                "conv_treatment": conv_t,
                "rate_control": rate_c,
                "rate_treatment": rate_t,
                "effect_hat": effect_hat,
                "se_hat": se_hat,
                "z_stat": z_stat,
                "p_value": p_value,
                "true_logodds_effect": theta_i,
            }
        )

    return pd.DataFrame.from_records(records)


sim_config = PortfolioSimConfig()
df_portfolio = simulate_experiment_portfolio(sim_config)
df_portfolio.head()


In [ ]:

df_portfolio[["effect_hat", "se_hat", "p_value"]].describe()



We now have a simulated **portfolio of experiments** with true heterogeneous effects,
observed effect estimates, standard errors and p-values.



## 2) Exploring the portfolio

We can look at:

- The distribution of estimated effects.  
- How often we see **significant** experiments at a given level (e.g., 5%).  
- How much **winner's curse** we get if we only look at significant winners.


In [ ]:

alpha = 0.05
sig_mask = df_portfolio["p_value"] < alpha

print("Total experiments:", len(df_portfolio))
print("Significant at 5%:", sig_mask.sum())
print("Share significant:", sig_mask.mean())

fig, axes = plt.subplots(1, 2, figsize=(10, 4))

axes[0].hist(df_portfolio["effect_hat"], bins=20)
axes[0].set_title("All experiments: effect_hat")
axes[0].set_xlabel("effect_hat (p_T - p_C)")
axes[0].set_ylabel("count")

axes[1].hist(df_portfolio.loc[sig_mask, "effect_hat"], bins=20)
axes[1].set_title("Significant experiments only")
axes[1].set_xlabel("effect_hat (p_T - p_C)")
axes[1].set_ylabel("count")

fig.suptitle("Distribution of observed effects (winner's curse demo)")
plt.tight_layout()
plt.show()



The right-hand histogram typically shows **inflated effect sizes** because we only look
at experiments that happened to be significant (winner's curse / publication bias).

Meta-analysis helps to recover a more realistic **typical effect size** across the whole
portfolio, including small and non-significant experiments.



## 3) Fixed-effect meta-analysis

The **fixed-effect** model assumes that all experiments share the *same* true effect
\(\theta\), and any differences we see are due only to sampling noise.

For experiment \(i\) with estimate \(\hat\theta_i\) and standard error \(s_i\):

- Weight: \(w_i = 1 / s_i^2\).  
- Pooled estimate:

\[
\hat\theta_{FE} = \frac{\sum_i w_i \hat\theta_i}{\sum_i w_i}.
\]

- Standard error: \(s_{FE} = 1 / \sqrt{\sum_i w_i}\).


In [ ]:

@dataclass
class MetaAnalysisResult:
    model: str
    theta_hat: float
    se: float
    ci_low: float
    ci_high: float
    tau2: float | None = None


def meta_fixed_effect(effect_hat: np.ndarray, se_hat: np.ndarray) -> MetaAnalysisResult:
    effect_hat = np.asarray(effect_hat, dtype=float)
    se_hat = np.asarray(se_hat, dtype=float)

    var_hat = se_hat ** 2
    w = 1.0 / var_hat

    w_sum = float(np.sum(w))
    theta_hat = float(np.sum(w * effect_hat) / w_sum)
    se = 1.0 / math.sqrt(w_sum)

    z = 1.96
    ci_low = theta_hat - z * se
    ci_high = theta_hat + z * se

    return MetaAnalysisResult(
        model="fixed_effect",
        theta_hat=theta_hat,
        se=se,
        ci_low=ci_low,
        ci_high=ci_high,
        tau2=None,
    )


fe_res = meta_fixed_effect(
    df_portfolio["effect_hat"].to_numpy(),
    df_portfolio["se_hat"].to_numpy(),
)
fe_res



## 4) Random-effects meta-analysis (DerSimonian–Laird)

The **random-effects** model assumes that each experiment has its own true effect
\(\theta_i\), drawn from a distribution:

\[
\theta_i \sim \mathcal{N}(\mu, \tau^2),
\]

where:

- \(\mu\) is the **overall mean effect** across experiments.  
- \(\tau^2\) is the **between-experiment variance** (heterogeneity).

We observe noisy estimates \(\hat\theta_i \sim \mathcal{N}(\theta_i, s_i^2)\).

The DerSimonian–Laird (DL) method estimates \(\tau^2\) using the **Q-statistic** and then
computes a weighted average with **augmented variances** \(s_i^2 + \tau^2\).


In [ ]:

def meta_random_effects_dl(
    effect_hat: np.ndarray,
    se_hat: np.ndarray,
) -> MetaAnalysisResult:
    effect_hat = np.asarray(effect_hat, dtype=float)
    se_hat = np.asarray(se_hat, dtype=float)

    var_hat = se_hat ** 2
    w_fe = 1.0 / var_hat
    w_fe_sum = float(np.sum(w_fe))
    theta_fe = float(np.sum(w_fe * effect_hat) / w_fe_sum)

    Q = float(np.sum(w_fe * (effect_hat - theta_fe) ** 2))
    df = effect_hat.size - 1
    c = w_fe_sum - np.sum(w_fe ** 2) / w_fe_sum
    tau2 = max(0.0, (Q - df) / c) if c > 0 else 0.0

    w_re = 1.0 / (var_hat + tau2)
    w_re_sum = float(np.sum(w_re))
    theta_re = float(np.sum(w_re * effect_hat) / w_re_sum)
    se_re = 1.0 / math.sqrt(w_re_sum)

    z = 1.96
    ci_low = theta_re - z * se_re
    ci_high = theta_re + z * se_re

    return MetaAnalysisResult(
        model="random_effects_DL",
        theta_hat=theta_re,
        se=se_re,
        ci_low=ci_low,
        ci_high=ci_high,
        tau2=tau2,
    )


re_res = meta_random_effects_dl(
    df_portfolio["effect_hat"].to_numpy(),
    df_portfolio["se_hat"].to_numpy(),
)
re_res



The random-effects model accounts for **between-experiment heterogeneity**. The estimated
\(\tau^2\) gives a sense of how much the true effects differ across experiments.



## 5) Forest plot of experiment estimates and pooled effect

A **forest plot** shows:

- Each experiment's point estimate and confidence interval.  
- The fixed-effect and/or random-effects pooled estimate at the bottom.

This makes heterogeneity very visible.


In [ ]:

def plot_forest(
    df: pd.DataFrame,
    fe_res: MetaAnalysisResult,
    re_res: MetaAnalysisResult,
    max_experiments: int = 30,
) -> None:
    df_plot = df.sort_values("effect_hat").head(max_experiments).copy()
    k = len(df_plot)

    y_positions = np.arange(k)

    effects = df_plot["effect_hat"].to_numpy()
    ses = df_plot["se_hat"].to_numpy()
    ci_low = effects - 1.96 * ses
    ci_high = effects + 1.96 * ses

    plt.figure(figsize=(8, 0.3 * k + 3))

    for y, lo, hi in zip(y_positions, ci_low, ci_high):
        plt.hlines(y, lo, hi)
    plt.plot(effects, y_positions, "o")

    plt.axvline(0.0, linestyle="--", linewidth=1)
    plt.axvline(fe_res.theta_hat, linestyle="-", linewidth=1.5)
    plt.axvline(re_res.theta_hat, linestyle=":", linewidth=1.5)

    plt.yticks(y_positions, df_plot["exp_id"])
    plt.xlabel("Effect (p_T - p_C)")
    plt.ylabel("Experiment id (subset)")
    plt.title("Forest plot of experiment effects (subset)")
    plt.tight_layout()
    plt.show()


plot_forest(df_portfolio, fe_res, re_res, max_experiments=25)



The vertical dashed line at 0 represents **no effect**.

The two vertical lines represent the **fixed-effect** and **random-effects**
pooled estimates. Experiments scattered widely around them indicate substantial
heterogeneity.



## 6) Bayesian-style shrinkage for experiment effects (partial pooling)

We can reuse the random-effects idea in a **Bayesian / partial pooling** view.

Assume:

- \(\hat\theta_i \sim \mathcal{N}(\theta_i, s_i^2)\) (approximate normal estimator).  
- \(\theta_i \sim \mathcal{N}(\mu, \tau^2)\), where \(\mu\) and \(\tau^2\) come from
  the random-effects meta-analysis (as an empirical Bayes step).

Conditionally on \(\mu\) and \(\tau^2\), the posterior for each \(\theta_i\) is Normal:

\[
\theta_i \mid \hat\theta_i \sim \mathcal{N}(m_i, v_i)
\]

with

\[
v_i = \left(\frac{1}{s_i^2} + \frac{1}{\tau^2}\right)^{-1}, \quad
m_i = v_i \left(\frac{\hat\theta_i}{s_i^2} + \frac{\mu}{\tau^2}\right).
\]

This produces **shrunken per-experiment effects** that:

- Move noisy experiments closer to the global mean.  
- Leave well-measured experiments closer to their raw estimates.


In [ ]:

@dataclass
class ShrinkageResult:
    df: pd.DataFrame
    mu: float
    tau2: float


def shrink_experiment_effects(
    df: pd.DataFrame,
    re_res: MetaAnalysisResult,
) -> ShrinkageResult:
    if re_res.tau2 is None or re_res.tau2 <= 0.0:
        df2 = df.copy()
        df2["effect_shrunken"] = df2["effect_hat"]
        df2["effect_sd_post"] = df2["se_hat"]
        return ShrinkageResult(df=df2, mu=re_res.theta_hat, tau2=0.0)

    tau2 = float(re_res.tau2)
    mu = float(re_res.theta_hat)

    effects = df["effect_hat"].to_numpy()
    ses = df["se_hat"].to_numpy()
    var = ses ** 2

    v_i = 1.0 / (1.0 / var + 1.0 / tau2)
    m_i = v_i * (effects / var + mu / tau2)

    df2 = df.copy()
    df2["effect_shrunken"] = m_i
    df2["effect_sd_post"] = np.sqrt(v_i)

    return ShrinkageResult(df=df2, mu=mu, tau2=tau2)


shrink_res = shrink_experiment_effects(df_portfolio, re_res)
shrink_res.df.head()


In [ ]:

df_s = shrink_res.df.sort_values("effect_hat").reset_index(drop=True)

plt.figure(figsize=(8, 5))
plt.scatter(df_s["effect_hat"], df_s["effect_shrunken"])
plt.axhline(shrink_res.mu, linestyle="--", linewidth=1)
plt.axvline(shrink_res.mu, linestyle="--", linewidth=1)
plt.xlabel("Raw effect_hat")
plt.ylabel("Shrunken effect")
plt.title("Partial pooling: raw vs shrunken experiment effects")
plt.tight_layout()
plt.show()



In the scatter plot:

- Points near the diagonal are experiments with **little shrinkage** (large sample sizes).  
- Points pulled towards the intersection of the dashed lines are **noisier experiments**
  that get pulled towards the global mean.

This is a simple empirical Bayes view of **partial pooling** for experiment effects.



## 7) Using meta-analysis to set priors and expectations

From the random-effects model we get two key quantities:

- \(\mu\): typical effect size across experiments.  
- \(\tau\): between-experiment standard deviation.

This can be used to set **realistic priors** for new experiments:

- Before starting a new test, treat its unknown true effect \(\theta_{\text{new}}\) as:

\[
\theta_{\text{new}} \sim \mathcal{N}(\mu, \tau^2)
\]

- This prior can then be used in Bayesian analyses of the new experiment, or simply
  as a sanity check when someone expects a huge uplift.


In [ ]:

mu = shrink_res.mu
tau = math.sqrt(max(0.0, shrink_res.tau2))

print("Meta-level mean effect (mu):", mu)
print("Meta-level between-experiment sd (tau):", tau)

if tau > 0:
    z = (0.0 - mu) / tau
    cdf = 0.5 * (1.0 + math.erf(z / math.sqrt(2.0)))
    prob_positive = 1.0 - cdf
else:
    prob_positive = 1.0 if mu > 0 else 0.0

print("P(theta_new > 0) under meta prior:", prob_positive)



You can expose these meta-level summaries to product / growth teams as:

- A **prior belief** about likely effect sizes.  
- A way to calibrate expectations: *"Most experiments are in the X–Y% range."*  
- Input into **power and MDE calculations** for future experiments.



## 8) Significance-filtered vs full meta-analysis

A common failure mode in real experimentation programs is to only pay attention to
**“successful”** experiments (e.g., p-value < 0.05).

Here we compare:

- Meta-analysis using **all experiments** (full portfolio).  
- Meta-analysis using only **statistically significant** experiments.

We expect the significance-filtered meta-analysis to be **biased upwards** because it
ignores all the noisy near-zero / negative experiments that did not pass the threshold.


In [ ]:

alpha_sig = 0.05
mask_sig = df_portfolio["p_value"] < alpha_sig

full_fe = meta_fixed_effect(
    df_portfolio["effect_hat"].to_numpy(),
    df_portfolio["se_hat"].to_numpy(),
)
full_re = meta_random_effects_dl(
    df_portfolio["effect_hat"].to_numpy(),
    df_portfolio["se_hat"].to_numpy(),
)

df_sig = df_portfolio.loc[mask_sig].copy()

if len(df_sig) > 0:
    sig_fe = meta_fixed_effect(
        df_sig["effect_hat"].to_numpy(),
        df_sig["se_hat"].to_numpy(),
    )
    sig_re = meta_random_effects_dl(
        df_sig["effect_hat"].to_numpy(),
        df_sig["se_hat"].to_numpy(),
    )
else:
    sig_fe = None
    sig_re = None

print("Number of experiments (full):", len(df_portfolio))
print("Number of significant experiments:", len(df_sig))
print("---")
print("Full fixed-effect:", full_fe)
print("Full random-effects:", full_re)
print("---")
print("Significant-only fixed-effect:", sig_fe)
print("Significant-only random-effects:", sig_re)


In [ ]:

# Quick visual comparison of pooled estimates (full vs significant-only)

labels = []
means = []
errors = []

labels.append("Full FE")
means.append(full_fe.theta_hat)
errors.append(1.96 * full_fe.se)

labels.append("Full RE")
means.append(full_re.theta_hat)
errors.append(1.96 * full_re.se)

if sig_fe is not None and sig_re is not None:
    labels.append("Sig-only FE")
    means.append(sig_fe.theta_hat)
    errors.append(1.96 * sig_fe.se)

    labels.append("Sig-only RE")
    means.append(sig_re.theta_hat)
    errors.append(1.96 * sig_re.se)

x = range(len(labels))

plt.figure(figsize=(8, 4))
plt.errorbar(x, means, yerr=errors, fmt="o")
plt.axhline(0.0, linestyle="--", linewidth=1)
plt.xticks(list(x), labels)
plt.ylabel("Pooled effect (p_T - p_C)")
plt.title("Pooled estimates: full portfolio vs significance-filtered")
plt.tight_layout()
plt.show()



In general you should meta-analyze **all experiments**, not just “winners”.

Filtering on significance before combining results tends to **inflate** the pooled
estimate and gives an over-optimistic view of what experiments usually deliver.



## 9) Bayesian hierarchical meta-analysis (simple version)

We now add a lightweight **Bayesian hierarchical view** on top of the random-effects
model.

Assume:

- Observed per-experiment estimates: \(y_i = \hat\theta_i\).  
- Observation model: \(y_i \sim \mathcal{N}(\mu, \tau^2 + s_i^2)\), where
  \(s_i\) is the standard error from the individual experiment and \(\tau^2\) is the
  between-experiment variance (heterogeneity).  
- Prior for the meta-mean: \(\mu \sim \mathcal{N}(m_0, s_0^2)\).

Conditionally on \(\tau^2\), the posterior for \(\mu\) is still Normal and has
closed form:

\[
v_{\mu} = \left( \frac{1}{s_0^2} + \sum_i \frac{1}{\tau^2 + s_i^2} \right)^{-1}, \quad
m_{\mu} = v_{\mu} \left( \frac{m_0}{s_0^2} + \sum_i \frac{y_i}{\tau^2 + s_i^2} \right).
\]

Here we use the DerSimonian–Laird estimate of \(\tau^2\) as an empirical Bayes plug-in.
A fully Bayesian treatment would also place a prior on \(\tau^2\) and sample it;
that typically requires MCMC and external libraries.


In [ ]:

@dataclass
class BayesMetaMuPosterior:
    mean: float
    sd: float
    prior_mean: float
    prior_sd: float
    tau2_used: float


def bayes_meta_mu_posterior(
    df: pd.DataFrame,
    re_res: MetaAnalysisResult,
    prior_mean: float = 0.0,
    prior_sd: float = 0.05,
) -> BayesMetaMuPosterior:
    """Posterior for the meta-mean mu under a Normal prior and known tau^2.

    Parameters
    ----------
    df : DataFrame
        Portfolio with columns ['effect_hat', 'se_hat'].
    re_res : MetaAnalysisResult
        Random-effects meta-analysis result providing tau^2 and a point estimate.
    prior_mean : float
        Prior mean m0 for mu.
    prior_sd : float
        Prior standard deviation s0 for mu.

    Returns
    -------
    BayesMetaMuPosterior
        Posterior mean and standard deviation for mu.
    """
    if re_res.tau2 is None:
        raise ValueError("Random-effects result must contain tau2.")

    tau2 = max(0.0, float(re_res.tau2))
    effects = df["effect_hat"].to_numpy()
    ses = df["se_hat"].to_numpy()
    var_obs = ses**2 + tau2

    m0 = prior_mean
    s0_sq = prior_sd**2

    # Posterior variance
    inv_v = 1.0 / s0_sq + np.sum(1.0 / var_obs)
    v_mu = 1.0 / inv_v

    # Posterior mean
    m_mu = v_mu * (m0 / s0_sq + np.sum(effects / var_obs))

    return BayesMetaMuPosterior(
        mean=float(m_mu),
        sd=float(math.sqrt(v_mu)),
        prior_mean=prior_mean,
        prior_sd=prior_sd,
        tau2_used=tau2,
    )


bayes_mu_post = bayes_meta_mu_posterior(
    df_portfolio,
    re_res=re_res,
    prior_mean=0.0,
    prior_sd=0.05,
)
bayes_mu_post


In [ ]:

# Compare FE, RE, and Bayesian meta-mean estimates

print("Fixed-effect pooled mean:", fe_res.theta_hat, "+/-", 1.96 * fe_res.se)
print("Random-effects pooled mean:", re_res.theta_hat, "+/-", 1.96 * re_res.se)
print(
    "Bayesian mu posterior mean:", bayes_mu_post.mean,
    "+/-", 1.96 * bayes_mu_post.sd,
)

# Simple visualization of the posterior for mu
mu_grid = np.linspace(
    bayes_mu_post.mean - 4 * bayes_mu_post.sd,
    bayes_mu_post.mean + 4 * bayes_mu_post.sd,
    200,
)
dens = (1.0 / (math.sqrt(2.0 * math.pi) * bayes_mu_post.sd)
        * np.exp(-0.5 * ((mu_grid - bayes_mu_post.mean) / bayes_mu_post.sd) ** 2))

plt.figure(figsize=(8, 4))
plt.plot(mu_grid, dens)
plt.axvline(0.0, linestyle="--", linewidth=1, label="no effect")
plt.axvline(bayes_mu_post.mean, linestyle="-", linewidth=1.5, label="posterior mean")
plt.xlabel("mu (meta-level mean effect)")
plt.ylabel("posterior density (up to scaling)")
plt.title("Posterior for meta-level mean effect mu")
plt.legend()
plt.tight_layout()
plt.show()



This Bayesian hierarchical view gives you:

- A posterior distribution for the **overall mean effect** \(\mu\) across experiments.  
- A way to quantify uncertainty around \(\mu\) (not just a point estimate and CI).  
- A prior that you can reuse for new experiments, together with the
  **per-experiment partial pooling** we implemented earlier.

In practice you would typically implement a full Bayesian hierarchical model with
a prior on \(\tau^2\) and sample both \(\mu\) and \(\tau^2\) using MCMC
(PyMC, Stan, etc.), but the structure here mirrors that model in a lightweight,
teachable way.



## 10) Meta-experiment dashboard

To make this notebook directly useful as a **portfolio summary**, we add a small
“meta-experiment dashboard” cell that pulls together the key quantities:

- Meta-level mean effect \(\mu\).  
- Between-experiment standard deviation \(\tau\).  
- Probability that a random future experiment has **positive effect**,  
  \(P(\theta_{\text{new}} > 0)\).  
- Uplift bands (e.g. 10%, 50%, 90% quantiles) for \(\theta_{\text{new}}\) under
  the meta prior.

This is the kind of summary you can show to stakeholders as a **“what do experiments
usually do here?”** overview.


In [ ]:

from dataclasses import asdict

def compute_meta_dashboard(
    shrink_res: ShrinkageResult,
    bayes_mu_post: BayesMetaMuPosterior,
) -> pd.DataFrame:
    """Build a small dashboard of meta-level quantities.

    Parameters
    ----------
    shrink_res : ShrinkageResult
        Result of shrink_experiment_effects, carrying mu and tau^2.
    bayes_mu_post : BayesMetaMuPosterior
        Posterior for the meta-level mean effect mu.

    Returns
    -------
    DataFrame
        One-row summary with key statistics.
    """
    mu = shrink_res.mu
    tau = math.sqrt(max(0.0, shrink_res.tau2))

    # Probability theta_new > 0 under Normal(mu, tau^2)
    if tau > 0:
        z0 = (0.0 - mu) / tau
        cdf0 = 0.5 * (1.0 + math.erf(z0 / math.sqrt(2.0)))
        prob_positive = 1.0 - cdf0
    else:
        prob_positive = 1.0 if mu > 0 else 0.0

    # Uplift bands: 10%, 50%, 90% quantiles of theta_new ~ N(mu, tau^2)
    from math import erf, sqrt

    def normal_ppf(q: float, mean: float, sd: float) -> float:
        """Approximate inverse CDF using binary search on Normal(mean, sd^2)."""
        # Simple, self-contained implementation; accuracy is enough for reporting.
        if sd <= 0:
            return mean
        low, high = mean - 10 * sd, mean + 10 * sd
        for _ in range(60):
            mid = 0.5 * (low + high)
            z = (mid - mean) / (sd * sqrt(2.0))
            cdf_mid = 0.5 * (1.0 + erf(z))
            if cdf_mid < q:
                low = mid
            else:
                high = mid
        return 0.5 * (low + high)

    if tau > 0:
        q10 = normal_ppf(0.10, mu, tau)
        q50 = normal_ppf(0.50, mu, tau)
        q90 = normal_ppf(0.90, mu, tau)
    else:
        q10 = q50 = q90 = mu

    data = {
        "meta_mu_mean": mu,
        "meta_tau_sd": tau,
        "P(theta_new > 0)": prob_positive,
        "theta_new_q10": q10,
        "theta_new_q50": q50,
        "theta_new_q90": q90,
        "bayes_mu_post_mean": bayes_mu_post.mean,
        "bayes_mu_post_sd": bayes_mu_post.sd,
    }
    return pd.DataFrame([data])


try:
    dashboard_df = compute_meta_dashboard(shrink_res, bayes_mu_post)
    display(dashboard_df)
except NameError as e:
    print(
        "Dashboard depends on 'shrink_res' and 'bayes_mu_post'. "
        "Make sure to run the previous meta-analysis cells first."
    )
